In [10]:
%pip install pandas sentence-transformers langchain_community


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   -------------------------------- ------- 184.3/227.1 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 227.1/227.1 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 16.3 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/11.0 MB 22.4 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.0 MB 15.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 18.2 MB/s eta 0:00:01
   -------------- -----------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ESTO FUNCIONA MUY BIEN

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama

def autocompletado(texto):
    # Inicializa el modelo de Ollama
    llm = Ollama(
        model="llama3:latest",
        system="Tienes que completar las oraciones que pasa el usuario de una forma que tenga sentido y sea concisa, incluyendo el texto que ingresa el usuario como el principio de la oracion completa. Es importante que la repuesta sea unicamente la oración completa, sin agregar información adicional."
    )

    # Lee el archivo CSV y extrae el contenido de interés
    csv_file = 'mati.csv'  # Reemplaza con la ruta a tu archivo CSV
    df = pd.read_csv(csv_file)

    # Supongamos que la columna 'text' contiene los textos de interés
    texts = df['text'].tolist()

    # Función para ajustar la entrada al modelo basado en los textos del CSV
    def generate_prompt(user_input, examples):
        prompt = f"Ingresa el texto: {user_input}\n\n"
        prompt += "Basado en los siguientes ejemplos, completa la oración de manera similar:\n\n"
        for example in examples:
            prompt += f"- {example}\n"
        return prompt

    # Obtiene el texto del usuario
    user_text = texto

    # Genera el prompt con ejemplos (usamos menos ejemplos para reducir el tiempo de respuesta)
    prompt = generate_prompt(user_text, texts[-5:])  # Usamos los últimos 5 ejemplos
    # prompt = generate_prompt(user_text, texts)  # Usamos TODOS LOS EJEMPLOS

    # Función para generar y recolectar respuestas
    def generate_responses(prompt, n=3):
        responses = []
        for _ in range(n):
            response = ""
            print(end=' ', flush=True)
            for chunk in llm.stream(prompt):
                for char in chunk:
                    print(char, end='', flush=True)
                    response += char
            responses.append(response)
            print("\n\n")  # Nueva línea al final de cada respuesta
        return responses

    # Genera tres respuestas
    responses = generate_responses(prompt, n=3)
    return responses


# Prueba de la función
autocompletado(input("Ingresa el texto: "))

 Llamados a la acción para involucrar a la audiencia en la promoción de la inclusión digital y fomentar prácticas más accesibles en línea.


 Llamados a la acción para inspirar a líderes y organizaciones a implementar prácticas más inclusivas en sus estrategias y comunicación.


 Llamados a la acción para fomentar la conciencia y la participación en la promoción de la inclusión.




In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from langchain_community.llms import Ollama

# Inicializa el modelo de Ollama
llm = Ollama(
    model="llama3:latest",
    system="Tienes que completar las oraciones que pasa el usuario de una forma que tenga sentido y sea concisa, incluyendo el texto que ingresa el usuario como el principio de la oracion completa. Es importante que la repuesta sea unicamente la oración completa, sin agregar información adicional."
)

# Lee el archivo CSV y extrae el contenido de interés
csv_file = 'mati.csv'  # Reemplaza con la ruta a tu archivo CSV
df = pd.read_csv(csv_file)

# Supongamos que la columna 'text' contiene los textos de interés
texts = df['text'].tolist()

# Función para ajustar la entrada al modelo basado en los textos del CSV
def generate_prompt(user_input, examples):
    prompt = f"Ingresa el texto: {user_input}\n\n"
    prompt += "Basado en los siguientes ejemplos, completa la oración de manera similar:\n\n"
    for example in examples:
        prompt += f"- {example}\n"
    return prompt

# Función para generar y recolectar respuestas
def generate_responses(prompt, n=3):
    responses = []
    for _ in range(n):
        response = ""
        for chunk in llm.stream(prompt):
            for char in chunk:
                response += char
        responses.append(response)
    return responses

# Función de actualización cuando el usuario escribe
def on_text_change(event):
    user_text = text_var.get()
    if user_text:
        prompt = generate_prompt(user_text, texts)
        responses = generate_responses(prompt, n=3)
        for i, response in enumerate(responses):
            response_vars[i].set(response)

# Configuración de la interfaz gráfica con tkinter
root = tk.Tk()
root.title("Autocompletado Dinámico")

mainframe = ttk.Frame(root, padding="10 10 10 10")
mainframe.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

text_var = tk.StringVar()
response_vars = [tk.StringVar() for _ in range(3)]

ttk.Label(mainframe, text="Ingresa el texto:").grid(column=1, row=1, sticky=tk.W)
text_entry = ttk.Entry(mainframe, width=50, textvariable=text_var)
text_entry.grid(column=2, row=1, sticky=(tk.W, tk.E))
text_entry.bind('<KeyRelease>', on_text_change)

for i in range(3):
    ttk.Label(mainframe, textvariable=response_vars[i]).grid(column=1, row=2+i, columnspan=2, sticky=(tk.W, tk.E))

for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)

root.mainloop()


: 